In [1]:
train_data = []

with open('features.train') as feat:
    for row in feat.readlines():
        row = row [:len(row)-1]
        row = row.split(' ')
        row1 = [x for x in row if x != '']
        row1[0] = float(row1[0])
        row1[1] = float(row1[1])
        row1[2] = float(row1[2])
        train_data.append(row1)

test_data = []

with open('features.test') as feat:
    for row in feat.readlines():
        row = row [:len(row)-1]
        row = row.split(' ')
        row1 = [x for x in row if x != '']
        row1[0] = float(row1[0])
        row1[1] = float(row1[1])
        row1[2] = float(row1[2])
        test_data.append(row1)

In [2]:
def versus_all(n, data_set):
    #splits data and classification info
    data = [[data_set[x][1], data_set[x][2]] for x in range(0, len(data_set))]
    classifier = [1 if x[0] == n else -1 for x in data_set]
    return classifier, data

def count_diff(list_1, list_2):
    count = 0
    for x in range(0, len(list_1)):
        if list_1[x] != list_2[x]:
            count += 1
    return count

In [3]:
from sklearn import svm
for vers in [1, 3, 5, 7, 9]:
    clf = svm.SVC(kernel="poly", C=.01, degree=2, gamma=1)
    true_class, points = versus_all(vers, train_data)
    clf.fit(points, true_class)  
    sup_vectors = clf.support_vectors_
    clf_guess = clf.predict(points)
    E_in = count_diff(clf_guess, true_class)/len(true_class)
    print(E_in)


0.014264161294746948
0.09024825126868742
0.07625840076807022
0.08846523110684405
0.08832807570977919


In [43]:
# 1 versus 5 Classification
from sklearn import svm

train_1v5 = []
for x in train_data:
    if (x[0] == 1 or x[0] == 5):
        train_1v5.append(x)
        
test_1v5 = []
class_1v5 = []

for x in test_data:
    if (x[0] == 1 or x[0] == 5):
        test_1v5.append(x[1:])
        if x[0] == 1:
            class_1v5.append(1)
        else:
            class_1v5.append(-1)            

for d in [2, 5]:
    clf = svm.SVC(kernel="poly", C=1, degree=d, gamma=1, coef0=1)
    true_class, points = versus_all(1, train_1v5)
    clf.fit(points, true_class)  
    sup_vectors = clf.support_vectors_
    E_in = count_diff(clf.predict(points), true_class)/len(true_class)
    E_out = count_diff(clf.predict(test_1v5), class_1v5)/len(class_1v5)
    print("Degree value:" + str(d))
    print("Support Vectors:" + str(len(sup_vectors)))
    print("E_in:"+ str(E_in))
    print("E_out:"+ str(E_out))
    

Degree value:2
Support Vectors:24
E_in:0.0032030749519538757
E_out:0.018867924528301886
Degree value:5
Support Vectors:21
E_in:0.0032030749519538757
E_out:0.02122641509433962


In [69]:
# 1 versus 5 Classification
from sklearn import svm
import numpy as np
from random import uniform

train_1v5 = []
for x in train_data:
    if (x[0] == 1 or x[0] == 5):
        train_1v5.append(x)

selection = [0, 0, 0, 0, 0]
avg_E_cv = 0
for i in range(0, 100):
    idx_array = [int(uniform(0, len(train_1v5))) for x in range(0, int(len(train_1v5)/10))]

    true_class, points = versus_all(1, train_1v5)

    fold_train = []
    fold_train_class = []
    fold_validation = []
    fold_validation_class = []

    for idx, point in enumerate(train_1v5):
        if idx in idx_array:
            fold_validation.append(points[idx])
            fold_validation_class.append(true_class[idx])
        else:
            fold_train.append(points[idx])
            fold_train_class.append(true_class[idx])

    cv = []
    c_values =[.0001, .001, .01, .1, 1]
    for c in c_values:
        clf = svm.SVC(kernel="poly", C=c, degree=2, gamma=1, coef0=1)
        clf.fit(fold_train, fold_train_class)  
        E_cv = count_diff(clf.predict(fold_validation), fold_validation_class)/len(fold_validation)
        cv.append(E_cv)
    
    selection[cv.index(min(cv))] += 1

print(selection)

[34, 54, 1, 3, 8]


In [72]:
from sklearn import svm
import numpy as np
from random import uniform

train_1v5 = []
for x in train_data:
    if (x[0] == 1 or x[0] == 5):
        train_1v5.append(x)

avg_E_cv = 0
for i in range(0, 1000):
    idx_array = [int(uniform(0, len(train_1v5))) for x in range(0, int(len(train_1v5)/10))]

    true_class, points = versus_all(1, train_1v5)

    fold_train = []
    fold_train_class = []
    fold_validation = []
    fold_validation_class = []

    for idx, point in enumerate(train_1v5):
        if idx in idx_array:
            fold_validation.append(points[idx])
            fold_validation_class.append(true_class[idx])
        else:
            fold_train.append(points[idx])
            fold_train_class.append(true_class[idx])

    clf = svm.SVC(kernel="poly", C=.001, degree=2, gamma=1, coef0=1)
    clf.fit(fold_train, fold_train_class)  
    E_cv = count_diff(clf.predict(fold_validation), fold_validation_class)/len(fold_validation)
    avg_E_cv += E_cv/1000
print(avg_E_cv)

0.004834448959557935


In [73]:
# 1 versus 5 Classification
from sklearn import svm

train_1v5 = []
for x in train_data:
    if (x[0] == 1 or x[0] == 5):
        train_1v5.append(x)
        
test_1v5 = []
class_1v5 = []

for x in test_data:
    if (x[0] == 1 or x[0] == 5):
        test_1v5.append(x[1:])
        if x[0] == 1:
            class_1v5.append(1)
        else:
            class_1v5.append(-1)            

for c in [0.01, 1, 100, 10000, 1000000]:
    clf = svm.SVC(kernel="rbf", C=c, gamma=1, coef0=1)
    true_class, points = versus_all(1, train_1v5)
    clf.fit(points, true_class)  
    E_in = count_diff(clf.predict(points), true_class)/len(true_class)
    E_out = count_diff(clf.predict(test_1v5), class_1v5)/len(class_1v5)
    print("C value:" + str(c))
    print("E_in:"+ str(E_in))
    print("E_out:"+ str(E_out))

C value:0.01
E_in:0.003843689942344651
E_out:0.02358490566037736
C value:1
E_in:0.004484304932735426
E_out:0.02122641509433962
C value:100
E_in:0.0032030749519538757
E_out:0.018867924528301886
C value:10000
E_in:0.0025624599615631004
E_out:0.02358490566037736
C value:1000000
E_in:0.0006406149903907751
E_out:0.02358490566037736
